# Reranking trong RAG
## Lost In The Middle
Khi LLM nhận một context quá dài và thông tin cần thiết, có liên quan đến câu hỏi nằm ở phần giữa của context đó, thì nó sẽ có xu hướng là bỏ qua văn bản đó.

Nên để giải quyết việc này, cần sắp xếp thông tin có liên quan ở phía đầu và phía cuối của context (nơi mà LLM sẽ tập trung vào nhiều hơn), và đưa thông tin không liên quan vào giữa context.

In [20]:
! pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 786.1 kB/s eta 0:00:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 839.8 kB/s eta 0:00:0000:0100:01


In [1]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.document_compressors import CohereRerank
import os

In [2]:
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1,
    chunk_overlap=0,
)

vectorstore = Chroma(persist_directory="./data" ,embedding_function=HF_EMBEDDING)

/home/tranhuy/Desktop/Project/cantho-public-services/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
vectorstore.similarity_search()

In [1]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
question = "thủ tục đăng ký lại giấy chứng sinh cho con cần làm giấy tờ gì không?"
docs = retriever.get_relevant_documents(question)
docs

NameError: name 'vectorstore' is not defined

In [4]:
tokens = 0
for doc in docs:
    tokens += len(doc.page_content.split(" "))
tokens

9141

In [11]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
reordered_docs

NotImplementedError: 

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import OpenAI
import os

os.environ["COHERE_API_KEY"] = ""
CohereRerank.update_forward_refs()
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    question
)

In [8]:
# Reranking by the prompt template
from sentence_transformers.util import cos_sim
scores = []
question_embedding = HF_EMBEDDING.embed_query(question)
for doc in reordered_docs:
    embedding = HF_EMBEDDING.embed_query(doc.page_content.split(":")[0])
    score = cos_sim(question_embedding, embedding)
    scores.append(score)
scores

[tensor([[0.6163]]),
 tensor([[0.7035]]),
 tensor([[0.6295]]),
 tensor([[0.5296]]),
 tensor([[0.6608]]),
 tensor([[0.6041]]),
 tensor([[0.5875]]),
 tensor([[0.6888]]),
 tensor([[0.6688]]),
 tensor([[0.6357]])]